In [1]:
# Get raw data
with open('input/10.txt', 'r') as f:
    rawinput = f.read().strip()

In [2]:
import numpy as np

In [3]:
# Part 1
def get_buttons(x,y):
    buttons = np.zeros((len(x), y), dtype=int)
    buttons[*zip(*[[i,k] 
                   for i,j in enumerate(x) 
                   for k in eval(f'[{j[1:-1]}]')])] = 1
    return buttons

manual = [[(z:=np.array([[int(k=='#') for k in j[0][1:-1]]])), 
           get_buttons(j[1:-1], z.shape[1]), 
           np.array([[int(k) for k in j[-1][1:-1].split(',')]])]
          for i in rawinput.split('\n') 
          for j in [i.split()]]

hlist = lambda x: x.astype('<U1').view(f'<U{x.shape[1]}').reshape(-1).tolist()

tot_n_press = 0
for req, buttons, _ in manual:
    poss = np.zeros_like(req)
    hist = hlist(poss)
    n_press = 0
    while True:
        n_press += 1
        poss = np.logical_xor(np.repeat(poss, buttons.shape[0], axis=0),
                              np.tile(buttons, (poss.shape[0], 1))).astype(int)
        if np.any(np.all(poss == req, axis=1)):
            break
        poss = poss[((h:=np.array(hlist(poss)))[s:=np.lexsort((np.arange(h.shape[0]), h))] 
                     != np.append([-1], h[s][:-1]))
                    [np.argsort(s)]]
        poss = poss[~np.isin(hlist(poss), hist)]
        hist += hlist(poss)
    tot_n_press += n_press

tot_n_press

491

In [206]:
# Part 2

In [1061]:
tot_n_press = 0

for _, buttons, req in manual[1:2]:
    buttons = buttons[np.argsort(np.min(np.where((z:=buttons 
                                                     * np.sum(buttons, axis=0, keepdims=True) 
                                                     + np.expand_dims(np.arange(buttons.shape[1])/10, 0)) >= 1,
                                                 z, 99), 
                                        axis=1))]

    poss = np.zeros_like(req)
    n_press = np.zeros((1), dtype=int)
    
    for b in range(buttons.shape[0]):
        b_ix = np.arange(buttons.shape[1])[buttons[b]==1]
        rptx = np.min(req[:, b_ix] - poss[:, b_ix], axis=1) + 1
        pr_add = (z:=np.where((y:=np.expand_dims(np.arange(np.max(rptx)), 0)) 
                              < np.expand_dims(rptx, 1), 
                              y, -1).reshape(-1))[z>=0]
        n_press = np.repeat(n_press, rptx) + pr_add
        poss = np.repeat(poss, rptx, axis=0) + buttons[b:b+1]*np.expand_dims(pr_add, 1)

        filt = np.all((np.sum(buttons[b+1:], axis=0, keepdims=True) > 0) | (poss == req), axis=1)
        n_press = n_press[filt]
        poss = poss[filt]
        
    tot_n_press += np.min(n_press)
    
tot_n_press

/home/gavin/venvs/base/lib/python3.12/site-packages/IPython/core/displayhook.py:292: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


np.int64(38)

In [ ]:
# def get_req_order(bt, targ, done=None, reqid=None):
#     if done is None:
#         done = np.zeros_like(bt[:1])
#         reqid = np.arange(bt.shape[0])
#     if bt.shape[0]:
#         return [reqid[(ix:=np.lexsort((targ, np.sum(bt & ~done, axis=1)))[0])].item(),
#                 *get_req_order(np.delete(bt, ix, axis=0), 
#                                np.delete(targ, ix), 
#                                done | bt[ix:ix+1],
#                                np.delete(reqid, ix))]
#     else:
#         return []

# def get_combos(lbuttons, lreq, known=None):
#     if known is None:
#         known=dict()
#     dkey = f'{len(lbuttons)}-{lreq}'
#     if dkey not in known:
#         if len(lbuttons)>1:
#             known[dkey] = [[i]+j
#                            for i in lbuttons[0] 
#                            if i >= max(0, lreq-sum([max(j) for j in lbuttons[1:]]))
#                            for j in get_combos(lbuttons[1:], lreq-i, known)]
#         else:
#             known[dkey] = [[lreq.item()]] if lreq in lbuttons[0] else []
#     return known[dkey]

In [229]:
# # Order in which to break down requirement amounts
# req_order = get_req_order(buttons.T, req[0])
# req_order
# # Maximum times each button can be pressed before exceeding a joltage requirement
# bmax = np.nanmin(np.where((z:=buttons*req),z,np.nan), axis=1).astype(int)
# bmax
# # Initialize table of possible button presses
# poss = -np.ones_like([bmax])
# poss
# l = 8
# bmax[(btix:=np.arange(buttons.shape[0])[buttons[:, l] == 1])], btix
# rawcombos = get_combos(bmax[(btix:=np.arange(buttons.shape[0])[buttons[:, l] == 1])], req[0, l])
# rawcombos[:5]
# poss = np.repeat(poss, len(rawcombos), axis=0)
# poss[:, btix] = rawcombos
# poss[:5]
# l = 5
# btix = np.arange(buttons.shape[0])[buttons[:, l] == 1]
# btix
# # for p in poss:
# p = poss[0]
# p
# poss.shape
# [get_combos([[p[i].item()] if p[i]>=0 else range(bmax[i]+1) 
#              for i in btix], 
#             req[0, l])
#  for p in poss]

In [5]:
rawinput.split('\n')[12]

'[#.#####.##] (3,4) (0,1,3,4,5,7,8,9) (1,2,3,4,5,6,7,9) (0,1,2,3,7) (0,1,2,3,5,7,8,9) (0,1,2,3,5,6,7,9) (0,1,3,4,6) (0,4,5,6,7,8,9) (1,3,4,9) (0,3,5,6,7,8) (1,2,3,4,7,8) {51,83,66,96,74,54,52,81,36,64}'

In [1067]:
sum([get_n_presses(buttons, req) for _, buttons, req in manual])


21121

In [1064]:
def get_n_presses(buttons, req):
    button_presses = np.zeros((buttons.shape[0], 1), dtype=int)
    hist = {tuple(button_presses.reshape(-1).tolist())}
    while (error:=np.sum((np.sum(buttons * button_presses, axis=0, keepdims=True) - req)**2)):
        new_bp = (np.stack([button_presses] * ((y:=button_presses.shape[0]) * 2))
                + np.append((z:=np.identity(y, dtype=int)), -z).reshape(y*2,y,1))
        new_bp = new_bp[np.all(new_bp >= 0, axis=(1,2)) 
                        & [len({tuple(i)}&hist)==0 for i in np.squeeze(new_bp, axis=2).tolist()]]
        new_err = np.sum((np.sum(np.expand_dims(buttons, 0) * new_bp, axis=1) - req)**2, axis=1)
        button_presses = new_bp[np.argmax(np.abs(error) - np.abs(new_err))]
        hist = hist | {tuple(button_presses.reshape(-1).tolist())}
    return np.sum(button_presses).item()

In [1063]:
np.sum(button_presses)

np.int64(38)

In [1045]:
error = np.sum((np.sum(buttons * button_presses, axis=0, keepdims=True) - req)**2)
new_bp = (np.stack([button_presses] * ((y:=button_presses.shape[0]) * 2))
            + np.append((z:=np.identity(y, dtype=int)), -z).reshape(y*2,y,1))
new_bp = new_bp[np.all(new_bp >= 0, axis=(1,2)) 
                & [len({tuple(i)}&hist)==0 for i in np.squeeze(new_bp, axis=2).tolist()]]
new_err = np.sum((np.sum(np.expand_dims(buttons, 0) * new_bp, axis=1) - req)**2, axis=1)
button_presses = new_bp[np.argmax(np.abs(error) - np.abs(new_err))]
hist = hist | {tuple(button_presses.reshape(-1).tolist())}

(button_presses, 
req, 
np.sum(buttons * button_presses, axis=0, keepdims=True), 
req - np.sum(buttons * button_presses, axis=0, keepdims=True),
new_err[np.argmax(np.abs(error) - np.abs(new_err))])

(array([[ 9],
        [ 5],
        [25],
        [13],
        [ 7],
        [ 8],
        [ 6],
        [10],
        [ 8],
        [ 2],
        [12]]),
 array([[51, 83, 66, 96, 74, 54, 52, 81, 36, 64]]),
 array([[51, 84, 65, 95, 75, 57, 51, 82, 36, 63]]),
 array([[ 0, -1,  1,  1, -1, -3,  1, -1,  0,  1]]),
 np.int64(16))

In [877]:
len(hist)

22248

In [638]:
req, np.sum(buttons * button_presses, axis=0, keepdims=True)

(array([[51, 83, 66, 96, 74, 54, 52, 81, 36, 64]]),
 array([[39, 82, 58, 82, 67, 63, 47, 76, 32, 63]]))